<a href="https://colab.research.google.com/github/regmul/Case-Study-Predicting-Medical-Insurance-Costs-with-Linear-Regression/blob/main/Medical_Insurance_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part A: Understanding the Business Problem**

**1. Why might it be important for an insurance company to predict medical costs in advance?**

  i. Fair and accurate premium rating to their customers.-if they under estimate cost, they lose money and over estimation leads to loss of customers.

  ii. Risk management. Predicting costs allows insurers to identify high-risk individuals or groups and plan ahead.

**2.  Which variables in the dataset are likely to have the strongest impact on insurance charges? Justify your answer before looking at the data.**

a. Age- older people are more likely to have chronic illnesses than the younger generation.

b. BMI- A high BMI often indicates obesity, which is strongly linked to conditions like diabetes, heart disease, and hypertension.

c. Smokers- smokers may suffer from  ilnesses like cancer, lung disease, heart issues. Smoking is one of the biggest risk factors for high medical costs Insurance companies may apply very high surcharges for smokers.

## **Part B: Data Exploration & Cleaning**

 **1. Load and inspect the dataset. What are the features and the target variable?**

 Target variable is charges


In [3]:
# import the necessary libraries
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
# upload the csv file
uploaded = files.upload()

Saving Medical-Insurance.csv to Medical-Insurance.csv


In [43]:
# read the file
df = pd.read_csv('Medical-Insurance.csv')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             2772 non-null   object 
 1   sex             2772 non-null   int64  
 2   BMI             2772 non-null   float64
 3   no_of_children  2772 non-null   int64  
 4   smoker          2772 non-null   object 
 5   region          2772 non-null   int64  
 6   charges         2772 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 151.7+ KB


In [45]:
df.head(20)

,age,sex,BMI,no_of_children,smoker,region,charges
0,19,1,27.900,0,1,3,16884.92400
1,18,2,33.770,1,0,4,1725.55230
2,28,2,33.000,3,0,4,4449.46200
3,33,2,22.705,0,0,1,21984.47061
4,32,2,28.880,0,0,1,3866.85520
5,31,1,25.740,0,?,4,3756.62160
6,46,1,33.440,1,0,4,8240.58960
7,37,1,27.740,3,0,1,7281.50560
8,37,2,29.830,2,0,2,6406.41070
9,60,1,25.840,0,0,1,28923.13692


In [47]:
print(df.columns)

Index(['age', 'sex', 'BMI', 'no_of_children', 'smoker', 'region', 'charges'], dtype='object')


In [48]:
# setting values for X and y
X = df[['age', 'sex', 'BMI', 'no_of_children', 'smoker', 'region']]
y = df['charges']


In [49]:
# Replacing special characters like ? with nan for cleaning
df["age"] = pd.to_numeric(df["age"], errors="coerce")
df["smoker"] = pd.to_numeric(df["smoker"], errors="coerce")


**2. Check for missing values or data quality issues. How would you handle them if they exist?**

In [50]:
df.isnull().sum()

,0
age,4
sex,0
BMI,0
no_of_children,0
smoker,7
region,0
charges,0


missing values are:

i. Age 4

ii. smoker 7

**Data cleaning**

In [51]:

# Fill NaN values in age with the mean of age
df["age"] = df["age"].fillna(df["age"].mean())

# Check if any NaN remains
print(df["age"].isnull().sum())


0


In [52]:

# Fill NaN values in smoker with mode
df['smoker'] = df['smoker'].fillna(df['smoker'].mode()[0])


# Check if any NaN remains
print(df["smoker"].isnull().sum())

0


In [53]:
print(df.isnull().sum())

age               0
sex               0
BMI               0
no_of_children    0
smoker            0
region            0
charges           0
dtype: int64


 **3. Perform a descriptive analysis:**

 What is the average insurance charge in the dataset?

 Which region has the highest average charges?

 Do smokers pay more on average than non-smokers?

In [54]:
# average insurance charge
avg_charges = df['charges'].mean()
print('Average insurance charge is',avg_charges)

Average insurance charge is 13261.369959046897


In [55]:
# region with the highest average charges
Highest_charges = df['charges'].max()
print('Highest insurance charge is',Highest_charges)

Highest insurance charge is 63770.42801


In [56]:
# Do smokers pay more on average than non-smokers?
Avg_charges_by_smoker = df.groupby('smoker')['charges'].mean()
print(Avg_charges_by_smoker)

smoker
0.0     8417.874411
1.0    32223.139764
Name: charges, dtype: float64


Non-smokers (0.0) pay around $8,417 on average.

Smokers (1.0) pay around $32,223 on average.

Smokers pay more in medical charges compared to non-smokers.
This strongly suggests that smoking is a major cost driver in medical expenses.

 ## **Part C: Building the Model**

**1. Which features would you include in a linear regression model to predict charges? Why?**

Age - older individuals generally incur higher medical costs.

Sex - some medical costs differ by gender.

BMI - higher BMI is linked to lifestyle diseases, raising charges.

Number of children - more dependents can increase medical expenses.

Smoker - smokers have significantly higher charges than non-smokers.

Region - healthcare costs vary across regions.

**2. Encode any categorical variables (e.g., sex , smoker ,region ). Which encoding method would you choose and why?**

a. Sex and smoker i will use Label Encoding because they contain binary categorical variables.

b. Region i will use One-Hot Encoding because it has multi-class categorical variables.

In [57]:
from sklearn.model_selection  import train_test_split


In [101]:
df_slr = df[['smoker']]
df_slr

,smoker
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
2767,0.0
2768,0.0
2769,1.0
2770,0.0


**3. Split the dataset into training and testing sets. What proportion would you use, and why?**

I would split the dataset into 80% training and 20% testing. The 80/20 split is widely used because it provides enough data for the model to learn patterns effectively (80%), while still reserving a sufficient portion (20%) to reliably evaluate how well the model generalizes to unseen data.

In [102]:
X_train,X_test,y_train,y_test= train_test_split(df_slr,df['charges'],test_size=0.20,random_state=230)

In [103]:
X_train

,smoker
721,0.0
151,0.0
349,0.0
1242,0.0
2122,1.0
...,...
1554,0.0
271,1.0
560,0.0
768,0.0


 **4. Fit a linear regression model on the training set. Write the regression equation in the form:
 charges=β0+β1age)+β2bmi)+β3children)+...+ϵ**

 charges=β0​+β1​(age)+β2​(BMI)+β3​(children)+β4​(sex)+β5​(smoker)+β6​(region)+ϵ

 ## **Part D: Model Evaluation**

 **1. Evaluate your model using metrics such as R², MAE, MSE, RMSE. What do these metrics tell you about your model?**

Simple linear regression

In [104]:
from sklearn.linear_model import LinearRegression

In [105]:
model = LinearRegression()

In [106]:
model.fit(X_train,y_train)

LinearRegression()

In [107]:
y_pred = model.predict(X_test)

In [108]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Predictions on test set
y_pred = model.predict(X_test)

# Evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("R²:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)


R²: 0.6369584830346857
MAE: 5349.501211452078
MSE: 49474781.24355589
RMSE: 7033.831192426776


## **Explanation**

**R²** = 0.74 (≈ 74%)

This means the model explains about 74% of the variation in medical charges using the features- age, BMI, children, sex, smoker, region.


**MAE** = 4159.51

On average, the model’s predictions are off by about $4,159.

Since medical charges can range widely from a few thousand to over $60k, this is moderate error.

**MSE** = 39,969,821

This is the average squared error.

Because errors are squared, big mistakes have more weight. The large value suggests some patients’ charges are very poorly predicted likely heavy smokers with very high charges.

**RMSE** = 6,322.17

This is the “average prediction error” in the same units as charges.

So, typically, the model is off by around $6,300 per prediction.

Since charges vary a lot, this error is not bad, but it shows the model struggles with extreme cases.




## Multiple LR

In [109]:
X = df[['age', 'sex', 'BMI', 'no_of_children', 'smoker', 'region']]


In [110]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [111]:
model = LinearRegression()

In [112]:
model.fit(X_train,y_train)

LinearRegression()

In [113]:
y_pred = model.predict(X_test)

In [114]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Predictions on test set
y_pred = model.predict(X_test)

# Evaluation metrics
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("R²:", r2)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)

R²: 0.7395779785261583
MAE: 4159.5072628500375
MSE: 39969821.125533335
RMSE: 6322.169020639462


**3. Does the model tend to overestimate or underestimate charges for any particular group (e.g., smokers, elderly)?**

1 Smoker has the largest positive coefficient. Smoking is the single biggest driver of higher medical costs.

2. BMI- For each unit increase in BMI, charges go up by a few hundred dollars. The heavier a person is, the more likely they are to incur higher medical costs.

3. Age- Each additional year of age increases charges by several hundred dollars. Older people tend to have higher medical expenses.

4. Older people tend to have higher medical expenses. Each additional child increases charges slightly. Having more dependents slightly raises expected costs, but not as much as smoking or age.

5. Sex (Male/Female)- Gender does not strongly influence charges compared to other factors.

6. Region- Where someone lives only has a minor effect on charges.Part E:

## **Part E: Business Insights & Recommendations**

 **1. Does the model tend to overestimate or underestimate charges for any particular group (e.g., smokers, elderly)**

 Smoker → Being a smoker adds the largest increase in medical charges.

Older Age → Each additional year increases expected charges.

High BMI  → Higher BMI is linked to higher medical costs.

More Children → Slightly raises charges because of dependent coverage.

Region  → Some regions may show slightly higher costs, but this effect is much smaller compared to smoking, age, and BMI.

Sex (male/female) → Usually has a minor effect compared to smoking and BMI.

**2. If management wanted to adjust insurance premiums, which customer factors should be considered most strongly?**

Management should focus most strongly on smoking status, age, and BMI when adjusting premiums, because these three factors explain the largest share of medical cost differences.

 **3. Suggest at least two ways the model could be improved in the future (e.g., adding new features, using different regression techniques).**

~ Add More Relevant Features

The current model only uses age, sex, BMI, children, smoker, and region.

Additional health-related variables could improve accuracy, e.g.:

Exercise habits, diet, alcohol consumption.

Pre-existing conditions (diabetes, hypertension, etc.).

Genetic/family medical history.

This would capture more of the true drivers of healthcare costs.

~ Use Advanced Regression/ML Techniques

Linear regression assumes a straight-line relationship, which may not always hold.

More flexible methods could better capture complex patterns:

Polynomial regression (to capture non-linear effects, e.g., costs rising sharply after age 50).

Regularized regression (Ridge/Lasso) to reduce overfitting.